In [2]:
import pandas as pd
dataset=pd.read_csv('./export/dataset.csv',index_col=0)

In [310]:
import pandas as pd
from sympy import *
df=pd.read_csv('./export/q3_4数学期望和多项式回归数据总表.csv',index_col=0)
waste_data=pd.read_excel('./dataset/附件4.xlsx',sheet_name='Sheet1',index_col='单品名称')['损耗率(%)']

result=pd.DataFrame(columns=['单品名称','R_max','W','P','Q','S(Q)'])

for typ in df['单品名称'].values:
    # 导入常数
    df0=df[df['单品名称']==typ].copy()
    ec,ec2,ec3=df0['E(C)'].values[0],df0['E(C^2)'].values[0],df0['E(C^3)'].values[0]
    k1,k2,k3=df0['k1'].values[0],df0['k2'].values[0],df0['k3'].values[0]
    waste=waste_data[typ]/100
    
    # 定义函数
    w=symbols('w')
    r=k1*ec3*w*(1+w)**2+k2*ec2*(1+w)+k3*ec*w
    
    # 求导&求解
    dr=diff(r,w)
    solutions=solve(dr,w,real=True)
    
    # 计算
    w_max=max(solutions,key=lambda x: re(x))
    if im(w_max) != 0:
        r_max=r.subs(w,3) if r.subs(w,3)>=r.subs(w,0) else r.subs(w,0)
        w_max=3 if r.subs(w,3)>=r.subs(w,0) else 0
    else:
        r_max=r.subs(w,w_max)
    p=ec*(1+w_max)
    q=(k1*p**2+k2*p+k3)
    sq=(k1*p**2+k2*p+k3)/(1-waste)
    
    # 打表
    row={'单品名称':typ,'R_max':r_max,'W':w_max,'P':p,'Q':q,'S(Q)':sq}
    result=pd.concat([result,pd.DataFrame(row,index=[0])],axis=0,ignore_index=True)


,单品名称,r_max,w,p,q
0,上海青,97.7594563035235,0.348105878614046,4.15108762142837,12.4353516051959
1,云南油麦菜,73.7776429331418,0.996304046609717,6.08154064759184,37.1091408567422
2,云南生菜,533.131229106223,0.0994409096266600,3.95985632420234,5.27276601029209
3,外地茼蒿,346.756783052603,3,26.5828,5.479961
5,木耳菜,235.533239822091,3,11.2,4.912727
6,洪湖藕带,179.940381898998,0.242259849943212,18.6007294111547,8.03013721655311
7,竹叶菜,1123.50116724439,3,14.2784,8.5104
8,紫茄子(2),1099.10542550442,3,15.9572,57.809324
9,红薯尖,87.6212082452765,0.415742474118100,4.56449531080416,25.8989050412406
10,苋菜,316.722349197389,3,11.3572,9.492974


In [ ]:
result[(result['w']>=0) & (result['q']>=2.5)]
#.to_csv('./export/q2_6问题二最终结论.csv')